# Demo Problem
### Carissa Mayo and Anna Rossen

In [ ]:
# we need the updated version of deepxde (deep learning package) for this demo
!pip install DeepXDE

In [ ]:
import deepxde as dde
import numpy as np


## ODE System to model some biological process

In [ ]:
def ode_system(x, y):
    """ODE system.
    dy1/dx = y2
    dy2/dx = -y1
    """

    y1, y2 = y[:, 0:1], y[:, 1:]
    dy1_x = dde.grad.jacobian(y, x, i=0)
    dy2_x = dde.grad.jacobian(y, x, i=1)

    return [dy1_x - y2, dy2_x + y1]


In [ ]:
#sets boundary on initial condition
#determines if you are at the initial condition (returns TRUE for t=0, FALSE otherwise)
def boundary(_, on_initial):
    return on_initial
  

## Reference functions

In [ ]:
# Here, sine and cosine are used to balance periodicity - think of the feature layer
def func(x):
    """
    y1 = sin(x)
    y2 = cos(x)
    """
    return np.hstack((np.sin(x), np.cos(x)))

In [ ]:
#time domain 
geom = dde.geometry.TimeDomain(0, 10)

#initial conditions ic1 and ic2 using geom, boundary, and initial function:
ic1 = dde.icbc.IC(geom, lambda x: 0, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: 1, boundary, component=1)
data = dde.data.PDE(geom, ode_system, [ic1, ic2], 35, 2, solution=func, num_test=100)

## Creating the network

In [ ]:
layer_size = [1] + [50] * 3 + [2] # fully connected neural network of depth 4 (3 hidden layers) and width 50
activation = "tanh" #hyperbolic tangent function used as activation function (sigma) in the features layer
initializer = "Glorot uniform" #generates initial value from a uniform distribution
net = dde.nn.FNN(layer_size, activation, initializer)

In [ ]:
model = dde.Model(data, net) #build the model
model.compile("adam", lr=0.001, metrics=["l2 relative error"]) # first training of the model
# the Adam optimizer - minimizes the loss function via gradient-based optimizer
# learning rate - how quickly the model updates the concepts it has learned

losshistory, train_state = model.train(iterations=20000)

#plots best trained result and loss history 
#finds when the loss history is lowest and that corrseponds to best trained result

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

# Try on your own!!! See what happens with...
## - a different activation function (maybe swish)
## - a longer (or shorter) run time
## - different system of ODEs 
## - different reference functions